In [112]:
import random
import os
import timeit
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [113]:
class Node(object):

    def __init__(self):
        self.predecessors = []
        self.successors = []

        self.renewable_resource_requirements = []

        self.start_time = None
        self.finish_time = None

        self.duration = None

        self.name = None

        self.es = None
        self.ls = None
        self.ef = None
        self.lf = None

        self.prio_value = None

        self.started = False
        self.finished = False

        self.scheduled = False

        self.selection_probability = None

    def __repr__(self):
        return str(self.name)

In [114]:
class Project(object):

    def __init__(self):

        self.renewable_resource_availability = []
        self.number_of_jobs = None
        self.number_of_nondummy_jobs = None
        self.number_of_renewable_resources = None
        self.nodes = dict()

        self.horizon = None
        self.population = []

    def read_project(self, instance_filepath, instance_name):

        full_path = os.path.join(instance_filepath, instance_name)

        with open(full_path, "r") as file:
            file = list(file)
            for line_index, line in enumerate(file):
                if "projects" in line:
                    # Get number of jobs
                    dummy = file[line_index + 1].split(" ")
                    self.number_of_jobs = int(dummy[-1])
                    self.number_of_nondummy_jobs = self.number_of_jobs - 2

                    # Get horizon
                    dummy = file[line_index + 2].split(" ")
                    self.horizon = int(dummy[-1])

                    # Get number of renewable resources
                    dummy = file[line_index + 4].split(" ")
                    self.number_of_renewable_resources = int(dummy[-4])

                    # Create nodes and store in dictionary 'nodes'
                    for i in range(self.number_of_jobs):
                        dummy = Node()
                        dummy.name = i
                        self.nodes[i] = dummy

                elif "PRECEDENCE RELATIONS:" in line:

                    starting_line_index = line_index + 2

                    for i in range(self.number_of_jobs):
                        dummy = file[starting_line_index + i]
                        dummy = dummy.rstrip()
                        dummy = dummy.split(" ")
                        dummy = list(filter(None, dummy))
                        dummy = [int(entry) for entry in dummy]
                        # print("entry", entry)
                        # print(dummy)
                        number_of_successors = dummy[2]
                        for j in range(number_of_successors):
                            # careful: here offset needed due to renaming of nodes: Starting from 0 -> n+1
                            successor_node_name = dummy[3 + j] - 1
                            self.nodes[i].successors.append(self.nodes[successor_node_name])

                    for node in self.nodes.values():
                        for succ in node.successors:
                            succ.predecessors.append(node)

                    for node in self.nodes.values():
                        node.predecessors = list(set(node.predecessors))

                elif "REQUESTS/DURATIONS:" in line:

                    starting_line_index = line_index + 3

                    for i in range(self.number_of_jobs):
                        dummy = file[starting_line_index + i]
                        dummy = dummy.rstrip()
                        # print(dummy)
                        dummy = dummy.split(" ")
                        # print(dummy)
                        dummy = list(filter(None, dummy))
                        dummy = [float(entry) for entry in dummy]
                        # print(dummy)+
                        self.nodes[i].duration = dummy[2]
                        # print("self.nodes[i].duration",self.nodes[i].duration)
                        for k in range(self.number_of_renewable_resources):
                            # print(k)
                            self.nodes[i].renewable_resource_requirements.append(int(dummy[3 + k]))
                        # print("resource",self.nodes[i].renewable_resource_requirements)

                elif "RESOURCEAVAILABILITIES:" in line:

                    starting_line_index = line_index + 2

                    dummy = file[starting_line_index]
                    dummy = dummy.rstrip()
                    dummy = dummy.split(" ")
                    dummy = list(filter(None, dummy))
                    dummy = [float(entry) for entry in dummy]

                    for availability in dummy:
                        self.renewable_resource_availability.append(availability)

    def solve_instance_via_ga(self, pop_size, number_of_generations, mutation_probability, sgs):

        incumbent = Individual()

        population = self.create_initial_population(pop_size)

        for indiv in population:
            sgs(indiv)

        for gen in range(number_of_generations):
            print("gen: " + str(gen))
            print("incumbent fitness: " + str(incumbent.fitness))
            print("--")

            offsprings = self.crossover(population)
            offsprings = self.mutate(offsprings, mutation_probability)

            population, incumbent = self.rank_and_reduce(population, offsprings, incumbent, sgs)

        print("finished GA with fitness of: " + str(incumbent.fitness))

    def create_initial_population(self, pop_size):

        self.forward_backward_scheduling()

        population = []

        # add one pure MIN-LFT-rule individual, can be extended by further priority rules if desired

        dummy = Individual()
        nodes_sorted_by_min_lft = list(self.nodes.values())
        nodes_sorted_by_min_lft.sort(key=lambda x: x.lf)

        dummy.activity_list = nodes_sorted_by_min_lft
        population.append(dummy)

        # Sampling Procedure based on min lft:
        for i in range(1, pop_size):
            dummy = Individual()

            unselected_nodes = list(self.nodes.values())
            selected_nodes = []

            starting_node = unselected_nodes[0]
            selected_nodes.append(starting_node)
            unselected_nodes.remove(starting_node)

            while unselected_nodes:
                possibles = []
                for node in unselected_nodes:
                    if all([predecessor in selected_nodes for predecessor in node.predecessors]):
                        possibles.append(node)

                max_lft = max([_.lf for _ in possibles])
                total = sum([(max_lft - _.lf + 1) for _ in possibles])

                for node in possibles:
                    node.selection_probability = (max_lft - node.lf + 1) / total

                selected_node = random.choices(possibles, [_.selection_probability for _ in possibles])[0]

                selected_nodes.append(selected_node)
                unselected_nodes.remove(selected_node)

            dummy.activity_list = selected_nodes
            population.append(dummy)

        # Also an idea: Do not base selection solely on MIN-LFT priority rule, but instead add more random selection.

        return population

    def forward_backward_scheduling(self):

        # forward pass:
        self.nodes[0].es = 0
        self.nodes[0].ef = 0

        for node in self.nodes.values():
            if node == self.nodes[0]:
                continue
            node.es = max([_.ef for _ in node.predecessors])
            node.ef = node.es + node.duration

        # backward pass:
        finish_node = self.nodes[self.number_of_jobs - 1]
        finish_node.lf = self.horizon
        finish_node.ls = self.horizon

        nodes = list(self.nodes.values())
        nodes.reverse()

        for node in nodes:
            if node == finish_node:
                continue
            node.lf = min([_.ls for _ in node.successors])
            node.ls = node.lf - node.duration

        return None

    def crossover(self, population):

        random.shuffle(population)
        mothers = population[:int(len(population) / 2)]
        fathers = population[int(len(population) / 2):]

        q = random.randint(1, self.number_of_nondummy_jobs - 1)

        offsprings = []

        for mother, father in zip(mothers, fathers):
            # create daughter:
            daughter = Individual()
            mother_input = mother.activity_list[:q]
            father_input = [_ for _ in father.activity_list if _ not in mother_input]
            daughter.activity_list = mother_input + father_input
            offsprings.append(daughter)

            # create son:
            son = Individual()
            father_input = father.activity_list[:q]
            mother_input = [_ for _ in mother.activity_list if _ not in father_input]
            son.activity_list = father_input + mother_input
            offsprings.append(son)

        return offsprings

    def mutate(self, offsprings, mutation_probability):

        indiv: Individual
        for indiv in offsprings:
            for index, activity in enumerate(indiv.activity_list[:-1]):
                if random.random() < mutation_probability:
                    store_current_activity_list = indiv.activity_list[:]
                    indiv.activity_list[index + 1], indiv.activity_list[index] = indiv.activity_list[index], \
                                                                                 indiv.activity_list[index + 1]
                    if not indiv.check_precedence_feasibility():
                        indiv.activity_list = store_current_activity_list

        return offsprings

    def rank_and_reduce(self, population, offsprings, incumbent, sgs):

        for indiv in offsprings:
            sgs(indiv)

        new_population = population + offsprings
        new_population.sort(key=lambda x: x.fitness)

        new_population = new_population[:len(population)]

        if new_population[0].fitness < incumbent.fitness:
            incumbent = new_population[0]

        return new_population, incumbent

    
    def parallel_SGS(self, individual, debug=False):

        individual: Individual

        # set prio-values based on activity_list and reset states:
        for node in self.nodes.values():
            node.prio_value = individual.activity_list.index(node)
            node.start_time = None
            node.finish_time = None
            node.started = False
            node.finished = False
            node.scheduled = False

        # create resource profile indexed by k and t
        R_kt = [[k] * self.horizon for k in self.renewable_resource_availability]
        print("R_kt", R_kt)
        actives = set()
        eligibles = [self.nodes[0]]
        scheduled_activities = set()
        # print("eligibles", eligibles)
        current_t = 0
        resource_profile_changes = []

        while len(scheduled_activities) != self.number_of_jobs:
            print("Len_scheduleactivities", len(scheduled_activities))
            started_nodes_in_iteration = set()

            while eligibles:
                # print("eligibles",eligibles)

                selected_node: Node = eligibles[0]

                selected_node.scheduled = True
                selected_node.start_time = current_t
                selected_node.finish_time = current_t + selected_node.duration
                selected_node.started = True
                print("selected_node", selected_node.finish_time)
                # selected_node.finished = True
                # print("self.number_of_renewable_resources", self.number_of_renewable_resources)
                print("renewable_resource_requirements[k]", selected_node.renewable_resource_requirements)
                for k in range(self.number_of_renewable_resources):
                    for t_iterator in range(int(current_t),int(current_t + selected_node.duration)):
                        R_kt[k][t_iterator] -= selected_node.renewable_resource_requirements[k]
                       
                resource_profile_changes = [_.finish_time for _ in self.nodes.values() if _.finish_time]
                resource_profile_changes = list(set(resource_profile_changes))
                resource_profile_changes.sort()
                # print("resource_profile_changes", resource_profile_changes)
                
                actives.add(selected_node)
                eligibles.remove(selected_node)
                scheduled_activities.add(selected_node)
                started_nodes_in_iteration.add(selected_node)
                # print("number_of_renewable_resources", number_of_renewable_resources)
                for eligible in eligibles[:]:
                    violation = False

                    for k in range(self.number_of_renewable_resources):
                        for t_iterator in range(int(current_t), int(current_t + eligible.duration)):
                            if eligible.renewable_resource_requirements[k] > R_kt[k][t_iterator]:
                                violation = True
                                eligibles.remove(eligible)
                                
                                break
                        if violation:
                            break

            #resource_profile_changes = [resource_profile_change for resource_profile_change in resource_profile_changes
                                        #if resource_profile_change > current_t]
            resource_profile_changes =  list(filter(lambda x: x>current_t, resource_profile_changes))
            finish_of_actives = [n.start_time + n.duration for n in actives]
            # print("finish_of_actives", finish_of_actives)
            print("resource_profile_changes", resource_profile_changes)
            current_t = min(resource_profile_changes + finish_of_actives)
            print("current_t", current_t)
            eligibles = []
            for node in scheduled_activities:
                eligibles.extend(node.successors)

            eligibles = list(set(eligibles))
            eligibles = [_ for _ in eligibles if _ not in scheduled_activities]
            eligibles = [_ for _ in eligibles if (all([pred.scheduled for pred in _.predecessors]))]

            eligibles = [_ for _ in eligibles if (all([pred.finish_time <= current_t for pred in _.predecessors]))]

            for eligible in eligibles[:]:
                violation = False

                for k in range(self.number_of_renewable_resources):
                    for t_iterator in range(int(current_t), int(current_t + eligible.duration)):
                        if eligible.renewable_resource_requirements[k] > R_kt[k][t_iterator]:
                            violation = True
                            eligibles.remove(eligible)
                            break
                    if violation:
                        break

            eligibles.sort(key=lambda x: x.prio_value)
            print("eligibles", eligibles)
            actives.clear()
            actives.update([_ for _ in scheduled_activities if (_.start_time <= current_t < _.finish_time)])

        individual.fitness = max([_.finish_time for _ in scheduled_activities])

        schedule = []

  
      # Generate a list of 120 random hexadecimal colors
        my_colors = ['#%02x%02x%02x' % (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for _ in range(120)]

        fig, ax = plt.subplots(figsize=(10, 5))

        # Sort the scheduled_activities by node.name+1
        scheduled_activities = sorted(scheduled_activities, key=lambda x: x.name)

        for i, node in enumerate(scheduled_activities):
            label = f'Job {i+1}'  # Modify the label to include job number
            ax.barh(label, node.finish_time - node.start_time, left=node.start_time, color=my_colors[i % len(my_colors)])  # Set color for each bar using modulo operation

        ax.set_xlabel('Time')
        ax.set_ylabel('Activity')
        ax.set_title('Gantt Chart')
        ax.set_xlim(0, current_t) # Set x-axis limit to match the total project duration

        # Create a color plate legend for job numbers and move it outside of the plot
        handles = [plt.Rectangle((0,0),1,1, color=my_colors[i % len(my_colors)]) for i in range(len(scheduled_activities))]
        labels = [f'Job {node.name+1}' for node in scheduled_activities]  # Update the labels to use node.name
        ax.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc='upper left')

        plt.show()



In [115]:
class Individual(object):

    def __init__(self):
        self.fitness = float('inf')
        self.activity_list = []
        self.start_times = dict()
        self.finish_times = dict()

    def restore_precedence_of_activity_list(self):
        pass

    def check_precedence_feasibility(self):

        violation = False

        for node in self.activity_list:
            if all([self.activity_list.index(pred) < self.activity_list.index(node) for pred in node.predecessors]):
                continue
            else:
                violation = True

        return not violation


In [ ]:
if __name__ == "__main__":
    start_time = timeit.default_timer()

    my_test_project = Project()
    instance_filepath = os.path.join(os.getcwd(), "Instances", "j120.sm")

    # my_test_project.read_project(instance_filepath, "test.sm")
    my_test_project.read_project(instance_filepath, "/content/project3.sm")

    # Call the solve_instance_via_ga function with parallel_SGS as the callback
    my_test_project.solve_instance_via_ga(1, 1, 0.05, my_test_project.parallel_SGS)
    # Call the function with the scheduled activities
    

   
    end_time = timeit.default_timer()
    run_time = end_time - start_time

    print("finished in seconds: " + str(run_time))
